# <font color=#3498DB> TUTORIAL

-------------------------------------------------
<br>

#### Ocean gliders have been used by the Integrated Marine Observing System (IMOS) since 2008. They allow the collection of high-resolution information in space (depth, longitude, latitude) and in time. 

#### The Australian National Facility for Ocean Gliders (ANFOG) is responsible for the operation and maintenance of the ocean glider fleet, whilst the Australian Ocean Data Network (AODN) is responsible for making glider data available for anyone to download through the portal. 

#### This Notebook is a guide on how to access and plot the ocean glider data.

<br>
<br>

### In this tutorial, you will:
<br>

* ## Search and download AODN glider data

* ## Load ANFOG data into python 

* ## Look through ANFOG netCDF file data and attributes

* ## Produce plots

* ## Save modified data as netCDF file

<br>
<br>

# <font color=#3498DB> Before Starting

------
<br>

## Install packages:

* ### xarray
* ### pandas
* ### matplotlib
* ### cmocean 
* ### cartopy
* ### netcdf4

<br>
<div style="background:#D1F2EB;padding:5px;">    
<h3>pip install <span style="color:blue">package </span>
        or 
<h3>conda install -c conda-forge <span style="color:blue">package
    </div>


# <font color=#3498DB> Search and download AODN data
______________    
    
<br>
<br>

<h3>ANFOG stands for Australian National Facility for Ocean Gliders.
<h3>You can download ANFOG netCDF files from the Australian Ocean Data Network (<a href="https://portal.aodn.org.au/">AODN website</a>).

<br>
<br>
<br>
<br>

### When on the AODN portal navigate and select the Heron 2015 deployment for download:

<br>

<div style="background:#D1F2EB;padding:5px;">    
<h4> Platform -> glider 
<h4> Dataset -> IMOS - 'Australian National Facility for Ocean Gliders (ANFOG) - delayed mode glider deployments' 
<h4> Deployment -> 'Heron20150521' 
</div>

# After download:

### filename: 
#### IMOS_-_Australian_National_Facility_for_Ocean_Gliders_(ANFOG)_-_delayed_mode_glider_deployments_source_files.zip
#### unzip -> IMOS_ANFOG_BCEOPSTUVN_20150521T004128Z_SL416_FV01_timeseries_END-20150530T131213Z.nc


# <font color=#3498DB> Load ANFOG data into python

### Now we are going to load in the ANFOG Heron Island netCDF file we just downloaded from the AODN
<br>

### To do this, and the other things we plan to do later, we need to load in some python packages:
<br>
<div style="background:#D1F2EB;padding:5px;">   
<h3><a href="http://xarray.pydata.org/en/stable/">xarray</a>: makes working with labelled multi-dimensional arrays simple, efficient, and fun!
<h3><a href="https://pandas.pydata.org/">pandas</a>: high-performance, easy-to-use data structures and data analysis tools for the Python programming language
<h3><a href="https://matplotlib.org/">matplotlib</a>: 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms

<h3><a href="https://matplotlib.org/cmocean/">cmocean</a>: perceptually uniform colormaps
<h3><a href="https://scitools.org.uk/cartopy/docs/latest/">cartopy</a>: designed for geospatial data processing in order to produce maps and other geospatial data analyses    
<h3><a href="https://unidata.github.io/netcdf4-python/netCDF4/index.html">netcdf4</a>: Python interface to the netCDF C library    
</div>
<br>
<br>


In [ ]:
# for loading data and calculations
import xarray as xr
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
import matplotlib as mat
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import cmocean
# for creating glider map
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

### Define name of file, load using xarray with one line of code

In [ ]:
filename = 'IMOS_ANFOG_BCEOPSTUVN_20150521T004128Z_SL416_FV01_timeseries_END-20150530T131213Z.nc'
glider_data = xr.open_dataset(filename)
# convert from data set to data frame for easier plotting later
glider_data_frame = xr.Dataset.to_dataframe(glider_data)

# <font color=#3498DB> Look through netCDF file data and attributes

### what is 'glider_data'?

In [ ]:
type(glider_data)

#### xarray.Dataset is xarray’s multi-dimensional equivalent of a DataFrame. 
#### It is a dict-like container of labeled arrays (DataArray objects) with aligned dimensions. 
#### It is designed as an in-memory representation of the data model from the netCDF file format.

### Take a look at variable names and global attributes

In [ ]:
glider_data

In [ ]:
type(glider_data_frame)

#### This is a pandas.DataFrame.

#### The DataFrame is indexed by the Cartesian product of index coordinates (in the form of a pandas.MultiIndex).

#### Other coordinates are included as columns in the DataFrame.

In [ ]:
glider_data_frame

In [ ]:
glider_data_frame['PSAL']

### Take a look at variable-specific attributes

In [ ]:
glider_data.TEMP

In [ ]:
glider_data.PSAL

In [ ]:
glider_data.DOX1

# <font color=#3498DB> Produce plots

### Select variables of interest from glider data

In [ ]:
TIME = glider_data.TIME
PSAL = glider_data.PSAL
O2_1 = glider_data.DOX1
CPHL = glider_data.CPHL
TEMP = glider_data.TEMP
DEPTH = glider_data.DEPTH

### create new QC variables and apply QC to data

#### In this example, we will apply QC to temperature and practical salinity
<br> 

<img src="IMOSQC.png" alt="QC table IMOS" width="400" height="400">


#### Temperature:

In [ ]:
TEMP_QC_index = glider_data.TEMP_quality_control
TEMP_QC_index_01 = TEMP_QC_index == 1
TEMP_QC = TEMP[TEMP_QC_index_01]

#### Practical salinity

In [ ]:
PSAL_QC_index = glider_data.PSAL_quality_control
PSAL_QC_index_01 = PSAL_QC_index == 1
PSAL_QC = PSAL[PSAL_QC_index_01]

### With glider data, sometimes you need to combine QC of more than one variable.

<br>
 
<h3><a href="https://en.wikipedia.org/wiki/Temperature%E2%80%93salinity_diagram">T-S diagram</a>: Each array - TEMP,PSAL - needs to have the same number of elements/length otherwise there is an error. 


In [ ]:
TS_QC_bool = TEMP_QC_index_01*PSAL_QC_index_01
PSAL_QC_TS = PSAL[TS_QC_bool]
TEMP_QC_TS = TEMP[TS_QC_bool]

<h3><a href="https://en.wikipedia.org/wiki/Time_series">timeseries</a>: Time must have the same number of elements/length as the variable of interest.

In [ ]:
TEMP_TIME_QC = TIME[TEMP_QC_index_01]

### Glider locations

In [ ]:
LONG_min = xr.DataArray.min(glider_data.LONGITUDE)
LONG_max = xr.DataArray.min(glider_data.LONGITUDE)
LAT_min = xr.DataArray.min(glider_data.LATITUDE)
LAT_max = xr.DataArray.min(glider_data.LATITUDE)

stamen_terrain = cimgt.Stamen('terrain-background')
fig = plt.figure(figsize=(10, 20))

# Create a GeoAxes in the tile's projection.
#ax1 = fig.add_subplot(1, 2, 1, projection=stamen_terrain.crs)
#ax2 = fig.add_subplot(1, 2, 2, projection=stamen_terrain.crs)

ax1 = plt.axes(projection=stamen_terrain.crs)  # standard axes
ax1.set_extent([LONG_min-2,LONG_max+2, LAT_min-2, LAT_max+2], crs=ccrs.PlateCarree())
ax1.add_image(stamen_terrain, 8)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
ax1.set_xticks([150,151,152,153,154,155], crs=ccrs.PlateCarree())
ax1.set_yticks([-23 -22 -20 -19 -18 -17], crs=ccrs.PlateCarree())
plt.rcParams.update({'font.size': 30})
ax1.plot(glider_data.LONGITUDE,glider_data.LATITUDE,marker='o',color='b',markersize=3,
        transform=ccrs.PlateCarree(),label='Glider data', linestyle='None')

### Create T-S diagram

In [ ]:
# create figure with specific size
plt.figure(figsize=(10,10))
# scatter TEMP vs. PSAL
plt.scatter(PSAL_QC_TS,TEMP_QC_TS)
# add axis labels
plt.xlabel('Practical Salinity')
plt.ylabel('Seawater temperature [deg C]')
# reduce number of ticks
locs, labels = plt.xticks()
plt.xticks(locs[::2])

### Create timeseries plot

In [ ]:
# define plot font family, weight, and size
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 35}

# create figure with set size
plt.figure(figsize=(15,7))
# plot TEMP  timeseries
plt.plot(TEMP_TIME_QC,TEMP_QC)
# create axis labels
plt.xlabel('Time')
plt.ylabel('Seawater Temperature [deg C]')
# set font that was defined above
plt.rc('font', **font)
# get locations and labels of x-axis ticks
locs, labels = plt.xticks()
# change the frequency of the xtick to every 2nd element in previous tick array (locs)
plt.xticks(locs[::2])
# change time python number to date string
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
# add a grid
plt.grid(True)

### Create transect plot of TEMP, PSAL, and CHL

In [ ]:
##############
# PSAL
##############

# create figure with specific size and 3 horizontal subplots
fig, ax = plt.subplots(3,1,figsize=(20,30))
# set subplot colormap using cmocean library of perceptually uniformap color maps
cmap_haline = cmocean.cm.haline
# define scatter settings here
shortcut = dict(s=20, c=glider_data_frame['PSAL'], 
                marker='o', edgecolor='none',cmap=cmap_haline,
                vmin=35.5,vmax=35.6)
# scatter time vs depth colored by PSAL
cs1 = ax[0].scatter(glider_data_frame.index, glider_data_frame['DEPTH'], **shortcut)
# flip y-axis 
ax[0].invert_yaxis()
# set subplot labels
ax[0].set_ylabel('Depth [m]')
ax[0].set_xlabel('Time')
# set subplot axis limits
ax[0].set_xlim(glider_data_frame.index[0], glider_data_frame.index[-1])
ax[0].set_ylim(210, -1)
# reduce frequency of Xticks to reduce text overlap
start, end = ax[0].get_xlim()
# every 2nd tick between start and end ticks
ax[0].xaxis.set_ticks(np.arange(start, end, 2))
# display date string rather than number
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
# change fontsize to size 26
for item in ([ax[0].title, ax[0].xaxis.label, ax[0].yaxis.label] +
              ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    item.set_fontsize(26)
# create corresponding colorbar and label
cbar = plt.colorbar(cs1,orientation='vertical',ax=ax[0])
cbar.ax.set_ylabel('Practical Salinity',fontsize=26)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

##############
# TEMP
##############

# set subplot colormap using cmocean library of perceptually uniformap color maps
cmap_thermal = cmocean.cm.thermal
# define scatter settings here
shortcut = dict(s=20, c=glider_data_frame['TEMP'], 
                marker='o', edgecolor='none',cmap=cmap_thermal,
                vmin=19,vmax=24.5)
# scatter time vs depth colored by PSAL
cs2 = ax[1].scatter(glider_data_frame.index, glider_data_frame['DEPTH'], **shortcut)
# flip y-axis 
ax[1].invert_yaxis()
# set subplot labels
ax[1].set_ylabel('Depth [m]')
ax[1].set_xlabel('Time')
# set subplot axis limits
ax[1].set_xlim(glider_data_frame.index[1], glider_data_frame.index[-1])
ax[1].set_ylim(210, -1)
# reduce frequency of Xticks to reduce text overlap
start, end = ax[1].get_xlim()
# every 2nd tick between start and end ticks
ax[1].xaxis.set_ticks(np.arange(start, end, 2))
# display date string rather than number
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
# change fontsize to size 26
for item in ([ax[1].title, ax[1].xaxis.label, ax[1].yaxis.label] +
              ax[1].get_xticklabels() + ax[1].get_yticklabels()):
    item.set_fontsize(26)
# create corresponding colorbar and label
cbar = plt.colorbar(cs2,orientation='vertical',ax=ax[1])
cbar.ax.set_ylabel('In situ temperature [$^\circ$C]',fontsize=26)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

##############
# CHL
##############

# set subplot colormap using cmocean library of perceptually uniformap color maps
cmap_algae = cmocean.cm.algae
# define scatter settings here
shortcut = dict(s=20, c=glider_data_frame['CPHL'], 
                marker='o', edgecolor='none',cmap=cmap_algae,
                vmin=0.5,vmax=1)
# scatter time vs depth colored by PSAL
cs2 = ax[2].scatter(glider_data_frame.index, glider_data_frame['DEPTH'], **shortcut)
# flip y-axis 
ax[2].invert_yaxis()
# set subplot labels
ax[2].set_ylabel('Depth [m]')
ax[2].set_xlabel('Time')
# set subplot axis limits
ax[2].set_xlim(glider_data_frame.index[1], glider_data_frame.index[-1])
ax[2].set_ylim(210, -1)
# reduce frequency of Xticks to reduce text overlap
start, end = ax[2].get_xlim()
# every 2nd tick between start and end ticks
ax[2].xaxis.set_ticks(np.arange(start, end, 2))
# display date string rather than number
ax[2].xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
# change fontsize to size 26
for item in ([ax[2].title, ax[2].xaxis.label, ax[2].yaxis.label] +
              ax[2].get_xticklabels() + ax[2].get_yticklabels()):
    item.set_fontsize(26)
# create corresponding colorbar and label
cbar = plt.colorbar(cs2,orientation='vertical',ax=ax[2])
cbar.ax.set_ylabel('Chlorophyll fluorescence [mg$^{-3}$]',fontsize=26)

### Save plot as PNG

In [ ]:
fig.savefig('TEMP_PSAL_CHL_Heron.png', bbox_inches='tight')

# <font color=#3498DB> Save modified data as netCDF file

In [ ]:
mean_TEMP = glider_data.mean(dim='TIME')

In [ ]:
mean_TEMP

In [ ]:
new_filename = 'mean_TEMP.nc'
mean_TEMP.to_netcdf(path=new_filename)

### load in newly created 'mean_TEMP' file to test

In [ ]:
test = xr.open_dataset(new_filename)

In [ ]:
test.TEMP

In [ ]:
test.CPHL